In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from ta import trend, momentum, volatility
from tqdm import tqdm
# from sklearn.ensemble import RandomForestClassifier

In [2]:
# Modified data preprocessing for your CSV structure
def load_and_preprocess(file_path):
    df = pd.read_csv(file_path)
    df.rename(
        columns={
            "tick_volume": "Volume",
            "real_volume": "RealVolume",
            "spread": "Spread",
        },
        inplace=True,
    )

    # Convert time to datetime and set as index
    df["time"] = pd.to_datetime(df["time"])
    df.set_index("time", inplace=True)

    # Feature engineering for missing technical indicators
    df["Price_Range"] = df["High"] - df["Low"]
    df["Typical_Price"] = (df["High"] + df["Low"] + df["Close"]) / 3

    return df

In [3]:
def set_tech_indicators(df: pd.DataFrame) -> pd.DataFrame:
    df["EMA_50"] = trend.ema_indicator(df["Close"], window=50)
    df["EMA_200"] = trend.ema_indicator(df["Close"], window=200)
    df["RSI"] = momentum.rsi(df["Close"], window=14)

    df["ATR"] = volatility.average_true_range(
        df["High"], df["Low"], df["Close"], window=14
    )
    df["BB_high"] = volatility.bollinger_hband(df["Close"], window=20)
    df["BB_low"] = volatility.bollinger_lband(df["Close"], window=20)

    # df = df.dropna()

    return df

In [4]:
def enhanced_trend_analysis(prices: pd.DataFrame) -> str:
    """Modified trend analysis using available data"""
    # prices["EMA_50"] = trend.ema_indicator(prices["Close"], window=50)
    # prices["EMA_200"] = trend.ema_indicator(prices["Close"], window=200)
    # prices["RSI"] = momentum.rsi(prices["Close"], window=14)

    trend_direction = "Neutral"
    if (
        prices["EMA_50"].iloc[-1] > prices["EMA_200"].iloc[-1]
        and prices["RSI"].iloc[-1] > 50
    ):
        trend_direction = "Uptrend"
    elif (
        prices["EMA_50"].iloc[-1] < prices["EMA_200"].iloc[-1]
        and prices["RSI"].iloc[-1] < 50
    ):
        trend_direction = "Downtrend"

    return trend_direction

In [5]:
def volatility_analysis(prices: pd.DataFrame) -> str:
    """Volatility analysis using available columns"""
    # prices["ATR"] = volatility.average_true_range(
    #     prices["High"], prices["Low"], prices["Close"], window=14
    # )
    # prices["BB_high"] = volatility.bollinger_hband(prices["Close"], window=20)
    # prices["BB_low"] = volatility.bollinger_lband(prices["Close"], window=20)

    volatility_status = (
        "High" if prices["ATR"].iloc[-1] > prices["ATR"].mean() else "Low"
    )
    return volatility_status

In [6]:
def ml_feature_engineering(prices):
    """Create features from available data"""
    features = pd.DataFrame(
        {
            "RSI": momentum.rsi(prices["Close"], window=14),
            "MACD": trend.macd(prices["Close"]),
            "Price_Range": prices["High"] - prices["Low"],
            "Spread": prices["Spread"],
            "Volume_Change": prices["Volume"].pct_change(),
            "ATR": volatility.average_true_range(
                prices["High"], prices["Low"], prices["Close"], window=14
            ),
        }
    )

    return features.dropna()

In [7]:
def improved_entry_logic(prices: pd.DataFrame) -> str:
    """Entry logic using available data"""
    trend_direction = enhanced_trend_analysis(prices)
    volatility_status = volatility_analysis(prices)

    entry_signal = "No Entry"
    if (
        trend_direction == "Uptrend"
        and prices["Close"].iloc[-1] > prices["BB_high"].iloc[-1]
        and volatility_status == "High"
    ):
        entry_signal = "Entry Long"

    elif (
        trend_direction == "Downtrend"
        and prices["Close"].iloc[-1] < prices["BB_low"].iloc[-1]
        and volatility_status == "High"
    ):
        entry_signal = "Entry Short"

    return entry_signal

In [8]:
def dynamic_risk_management(prices):
    """Risk management using available volatility measures"""
    atr = prices["ATR"].iloc[-1]
    position_size = 0.02 * prices["Close"].iloc[-1] / atr
    position_size = np.clip(position_size, 0.01, 0.05)

    return position_size

In [9]:
def enhanced_backtest(df, signals):
    """Backtest adapted to available data"""
    df["position_size"] = np.nan
    df["PnL"] = np.nan
    # df["ATR"] = volatility.average_true_range(
    #     df["High"], df["Low"], df["Close"], window=14
    # )

    in_trade = False
    entry_price = 0
    position_size = 0

    for i in tqdm(range(len(df)), desc="Backtesting"):
        if signals[i] != 0 and not in_trade:
            position_size = dynamic_risk_management(df.iloc[: i + 1])
            entry_price = df["Close"].iloc[i]
            in_trade = True
            trade_direction = signals[i]

        elif in_trade:
            if trade_direction == 1:
                stop_loss = entry_price - 2 * df["ATR"].iloc[i]
                take_profit = entry_price + 3 * df["ATR"].iloc[i]

                if df["Low"].iloc[i] <= stop_loss:
                    df.at[df.index[i], "PnL"] = (
                        stop_loss - entry_price
                    ) * position_size
                    in_trade = False
                elif df["High"].iloc[i] >= take_profit:
                    df.at[df.index[i], "PnL"] = (
                        take_profit - entry_price
                    ) * position_size
                    in_trade = False

            elif trade_direction == -1:
                stop_loss = entry_price + 2 * df["ATR"].iloc[i]
                take_profit = entry_price - 3 * df["ATR"].iloc[i]

                if df["High"].iloc[i] >= stop_loss:
                    df.at[df.index[i], "PnL"] = (
                        entry_price - stop_loss
                    ) * position_size
                    in_trade = False
                elif df["Low"].iloc[i] <= take_profit:
                    df.at[df.index[i], "PnL"] = (
                        entry_price - take_profit
                    ) * position_size
                    in_trade = False

    return df

In [10]:
# Modified trading algorithm
def improved_trading_algorithm(price_data):
    price_data = set_tech_indicators(price_data)
    signals = np.full(len(price_data), 0)
    # Hold = 0, Buy = 1, Sell = -1
    for index in tqdm(range(len(price_data)), desc="Trading Algorithm Progress"):
        # if index < 200:  # Warm-up period for indicators
        #     continue

        current_data = price_data.iloc[: index + 1]
        entry_signal = improved_entry_logic(current_data)

        if entry_signal == "Entry Long":
            signals[index] = 1
        elif entry_signal == "Entry Short":
            signals[index] = -1
        else:
            signals[index] = 0

    return signals

In [11]:
# Updated statistics calculation
def get_stats(complete_trades):
    n_trades = len(complete_trades)
    win_rate = (
        len(complete_trades[complete_trades["PnL"] > 0]) / n_trades
        if n_trades > 0
        else 0
    )
    loss_rate = 1 - win_rate
    avg_win = (
        complete_trades[complete_trades["PnL"] > 0]["PnL"].mean() if n_trades > 0 else 0
    )
    avg_loss = (
        complete_trades[complete_trades["PnL"] < 0]["PnL"].mean() if n_trades > 0 else 0
    )
    expectancy = (win_rate * avg_win) - (loss_rate * abs(avg_loss))

    return n_trades, win_rate, loss_rate, avg_win, avg_loss, expectancy

In [12]:
# Main execution
file_path = "Q1-24M5.csv"  # Replace with your file path
df = load_and_preprocess(file_path)
signals = improved_trading_algorithm(df)

Trading Algorithm Progress: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 17315/17315 [00:13<00:00, 1268.05it/s]


In [13]:
backtest_result = enhanced_backtest(df, signals)

Backtesting: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 17315/17315 [00:00<00:00, 47207.36it/s]


In [16]:
# Plotting and statistics
complete_trades = backtest_result[~backtest_result["PnL"].isna()]
stats = pd.DataFrame(
    index=["n_trades", "win_rate", "loss_rate", "avg_win", "avg_loss", "expectancy"]
)
stats["Complete Trades"] = get_stats(complete_trades)
print(stats)

            Complete Trades
n_trades         179.000000
win_rate           0.357542
loss_rate          0.642458
avg_win            0.269247
avg_loss          -0.165427
expectancy        -0.010013
